In [ ]:
!pip install opencv-python requests
!sudo apt install imagemagick -y

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
# Check if GPUs are enabled for training
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
data_dir = './PokemonData'

In [ ]:
# prepare data
!chmod u+x mogrifyScript.sh
!./mogrifyScript.sh
!python3 skewData.py

In [ ]:
# Set variables to reformat img heights and width and set batch size.
img_height,img_width=300,300
batch_size=32

In [ ]:
# Create a training dataset with 80% of the data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  label_mode="int",
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# Create a validation dataset with 20% of the data
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  label_mode="int",
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# View classnames and check if images are loaded properly
class_names = train_ds.class_names
print(class_names)

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# enable multi-gpu
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


with strategy.scope():
    model = Sequential()
    # import pretrained model and output layer (where learning will take place)
    pretrained_model= tf.keras.applications.vgg19.VGG19(include_top=False,
                       input_shape=(300,300,3),
                       pooling='avg',classes=150,
                       weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False

    model.add(pretrained_model)

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(150, activation='softmax'))
    model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
model.save(os.path.join('./results','pokemonclassifierVGG19.h5'))
tf.saved_model.save(model, './results/saved_model')

In [ ]:
from keras.models import load_model
inference_model = load_model('./results/pokemonclassifierVGG19.h5')
# inference_model = model

In [ ]:
# download image from the web to perform inference
pic_url = "https://www.kindpng.com/picc/m/17-176253_transparent-kabuto-png-pokemon-kabutops-mega-evolution-png.png"
with open('pic1.jpg', 'wb') as handle:
    response = requests.get(pic_url, stream=True)

    if not response.ok:
        print(response)

    for block in response.iter_content(1024):
        if not block:
            break

        handle.write(block)

In [ ]:
import cv2
import pathlib
import requests
# Perform inference with training data

# inference_dir = pathlib.Path(data_dir)
# selected_pokemon = list(inference_dir.glob('Golem/*'))[0]

# Perform Inference with new data
selected_pokemon = pathlib.Path("./pic1.jpg")
PIL.Image.open(str(selected_pokemon))

image=cv2.imread(str(selected_pokemon))
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)

pred=inference_model.predict(image)
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)